In [ ]:
import numpy as np
from scipy.stats import cauchy,norm,uniform
from datetime import datetime
import matplotlib.pyplot as plt
import csv
import sys
sys.path.insert(0,'C:/Users/patip/Documents/Python/Algorithm')
from Benchmark.functionbenchmark import Salomon as myfunc

In [ ]:
dimention = 50
populationsize = 100
lowerbound = -100
upperbound = 99
round = 20000
u_CR,u_F = 0.5,0.5
scale_factor,cr_rate = 0,0
sf_set = []
sc_set = []
set_A = []
value = np.zeros(populationsize)
minimized_vector = np.array(dimention)
minimized_value_keep = np.zeros((round,3))
minimized_value = 0
population = np.zeros((populationsize,dimention))

for i in range(populationsize):
    dummy = np.random.uniform(low=lowerbound,high=upperbound+1,size=dimention)
    population[i:] = dummy



In [ ]:
def build_trial(sf:float,cr:float,r_vector1:np.array,r_vector2:np.array,target:np.array,best_vector:np.array)->np.array:
    dummy_vector= np.random.uniform(low=0,high=1,size=dimention)
    sf = cauchy.rvs(u_F,0.1)
    #print(u_F," : ",sf)
    cr = norm.rvs(u_CR,0.1)
    #print(u_CR," : ",cr)
    #print("////////////////////////////////")
    mutant_vector = target+sf*(best_vector-target)+sf*(r_vector1-r_vector2)
    trial = np.zeros(dimention)
    for i in range(dimention):
        if dummy_vector[i] > cr:
            trial[i] = target[i]
        else:
            trial[i] = mutant_vector[i]
    return trial,sf,cr

def eveluate(fun,arg)->float:
    return fun(arg)

def selection(sf:float,cr:float,trial_vector:np.array,target_vector:np.array,func)->np.array:
    trial_value = eveluate(func,trial_vector)
    target_value = eveluate(func,target_vector)
    if trial_value <= target_value:
        sf_set.append(sf)
        sc_set.append(cr)
        set_A.append(target_vector)
        return trial_vector
    else:
        return target_vector


In [ ]:
from tqdm.notebook import tqdm_notebook
lowerbound = -100
upperbound = 100
for k in range(1,11):
    u_CR,u_F = 0.5,0.5
    scale_factor,cr_rate = 0,0
    sf_set = []
    sc_set = []
    set_A = []
    population = np.zeros((populationsize,dimention))
    for i in range(populationsize):
        dummy = np.random.uniform(low=lowerbound,high=upperbound+1,size=dimention)
        population[i:] = dummy
    

    for i in range(populationsize):
        value[i] = eveluate(myfunc,population[i])
    min_val = min(value)
    index = np.where(value == min_val)[0][0]
    minimized_vector = population[index]

    count = 0
    index_rand1,index_rand2 =0,0
    now = datetime.now()
    for i in tqdm_notebook(range(round)):
        for j in range(populationsize):
            vectorrand = np.random.permutation(populationsize)
            vectorrand = vectorrand[vectorrand!=j]   #keep the element is not equal j
            if len(set_A) == 0:
                index_rand1 = vectorrand[0]
                index_rand2 = vectorrand[1]
                trial_vector,scale_factor,cr_rate = build_trial(sf=scale_factor,cr=cr_rate,r_vector1=population[index_rand1],r_vector2=population[index_rand2],target=population[j],best_vector=minimized_vector)
            else:
                index_rand1 = vectorrand[0]
                dummy_population = np.delete(population,index_rand1,0)
                value_j = np.where(dummy_population == population[j])[0][0]
                dummy_population = np.delete(dummy_population,value_j,0)
                A = np.array(set_A)
                p_u_a = np.concatenate((dummy_population,A),axis=0)
                index_rand2 = np.random.permutation(len(p_u_a))[0]
                trial_vector,scale_factor,cr_rate = build_trial(sf=scale_factor,cr=cr_rate,r_vector1=population[index_rand1],r_vector2=p_u_a[index_rand2],target=population[j],best_vector=minimized_vector)
            #print(scale_factor," : ",cr_rate)
            #print("||||||||||||||||||||||||||||||||")
            population[j] = selection(sf=scale_factor,cr=cr_rate,trial_vector=trial_vector,target_vector=population[j],func=myfunc)
            value[j] = eveluate(myfunc,population[j])

        now2 = datetime.now()
        if i == 0:
            minimized_value = min(value)
            count = 0
            index = np.where(value == minimized_value)[0][0]
            minimized_vector = population[index]
            minuted = (now2-now).seconds + ((now2-now).microseconds)*1E-6
            min_round = i
            minimized_value_keep[i][0] = minimized_value
            minimized_value_keep[i][1] = min_round
            minimized_value_keep[i][2] = minuted

        else:
            if minimized_value > min(value):
                count = 0
                minimized_value = min(value)
                index = np.where(value == minimized_value)[0][0]
                minimized_vector = population[index]
                minuted = (now2-now).seconds + ((now2-now).microseconds)*1E-6
                min_round = i
                minimized_value_keep[i][0] = minimized_value
                minimized_value_keep[i][1] = min_round
                minimized_value_keep[i][2] = minuted

            else:
                count = count+1
                minimized_value_keep[i][0] = minimized_value
                minimized_value_keep[i][1] = min_round
                minimized_value_keep[i][2] = minuted

        if count >= 5000:
            break
        c = uniform.rvs()
        dummy_sf = np.array(sf_set)
        dummy_sc = np.array(sc_set)

        #print(len(dummy_sf))
        #print(len(dummy_sc))
        sum_sc = np.sum(dummy_sc)
        sum_f = np.sum(dummy_sf)
        sum_sqare = np.sum(dummy_sf**2)
        #print(dummy_sf)
        #print(scale_factor)
        #print(sum_sqare)
        try:
            u_CR = (1-c)*u_CR+c*(sum_sc/len(dummy_sc))
            u_F = (1-c)*u_F+c*(sum_sqare/sum_f)
        except:
            u_CR,u_F = 0.5,0.5
        #print(u_CR)
        #print(u_F)
        set_A = []
        sf_set = [] 
        sc_set = []

    file_path = 'C:/Users/patip/Documents/Python/Algorithm/raw_reault/Salomon_result/JADE_Salomon_'+str(dimention)+'_run_'+str(k)+'.csv'
    with open(file_path,'w',encoding='UTF8',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Solution','round','time'])
        writer.writerows(minimized_value_keep)





